# Merge stagioni
Le stagioni che ho nella cartella, contenenti tutti i vari match giocati, vengono analizzate singolarmente:
- vengono divise le partite giocate in casa da quelle in trasferta
- vengono combinate le partite 

Infine si otterrà un dataset dato dalla differenza delle statistiche dei vari match che vedrà tutte le stagioni combinate. I dataset sono ordinati per data.

In [8]:
"""from os import listdir
from os.path import isfile, join
import pandas as pd
from analysis import MatchAnalysis

mypath = 'files/Serie A/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
name_csv_statistics_FE_season = [x for x in onlyfiles if 'statistics.csv' in x] 


merged_statistics = pd.DataFrame()

def get_season(season):
    years = season.split('_')
    yearL, yearR = '20'+years[0], '20'+years[1] 
    return yearL + '-' + yearR

array_ma = []
for name_statistics_single_season in sorted(name_csv_statistics_FE_season):
    print(mypath+name_statistics_single_season)
    statistics_single_season = pd.read_csv(mypath+name_statistics_single_season, index_col=0)

    #setto il nome della stagione come i 5 primi caratteri del nome del file
    statistics_single_season['season'] = name_statistics_single_season[:5]
    merged_statistics = merged_statistics.append(statistics_single_season)

merged_statistics.to_csv("files/Serie A/20_23 statistics.csv")
"""

'from os import listdir\nfrom os.path import isfile, join\nimport pandas as pd\nfrom analysis import MatchAnalysis\n\nmypath = \'files/Serie A/\'\nonlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]\nname_csv_statistics_FE_season = [x for x in onlyfiles if \'statistics.csv\' in x] \n\n\nmerged_statistics = pd.DataFrame()\n\ndef get_season(season):\n    years = season.split(\'_\')\n    yearL, yearR = \'20\'+years[0], \'20\'+years[1] \n    return yearL + \'-\' + yearR\n\narray_ma = []\nfor name_statistics_single_season in sorted(name_csv_statistics_FE_season):\n    print(mypath+name_statistics_single_season)\n    statistics_single_season = pd.read_csv(mypath+name_statistics_single_season, index_col=0)\n\n    #setto il nome della stagione come i 5 primi caratteri del nome del file\n    statistics_single_season[\'season\'] = name_statistics_single_season[:5]\n    merged_statistics = merged_statistics.append(statistics_single_season)\n\nmerged_statistics.to_csv("files/Serie 

In [9]:
from temp.football_predictions import FootballPredictions
from ranking import Ranking
from analysis import MatchAnalysis

"""r = Ranking("Serie A", '2022-2023')
r.read_matches(seasons=5)
ma = MatchAnalysis("files/Serie A/all stats.csv")
ma.set_ranking(r)
"""
#ma.divide_and_merge_home_away()
#ma.create_team_dataset()


In [10]:
#ma.split_and_set_avg(True, 5)
ma = MatchAnalysis("files/Serie A/all stats.csv")
ma.read_diff_dataset("diff_with_ranking.csv")

In [11]:
fp = FootballPredictions(ma.diff_dataset)
fp.read_all_predictions(False)
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates()

2019-08-30 Bologna SPAL
2020-07-12 Genoa SPAL
2020-07-16 SPAL Inter
2020-07-29 Hellas Verona SPAL
2020-08-02 SPAL Fiorentina
2022-03-06 Genoa Empoli


In [12]:
from my_tokenizer import MyTokenizer
mt = MyTokenizer(fp.df)
#mt.feature_normalization()
mt.clean_text()

X_train, X_test, y_train, y_test = mt.set_bigram_and_get_sets()

In [16]:
from classification import Classification
c = Classification(X_train, X_test, y_train, y_test) 
model = c.open_model('Decision Tree stats.model')

In [ ]:
mt.set_label_prediction(model.y_pred)

In [26]:
ma.diff_dataset.drop(columns=['completed_passings','total_passings', 'corners'], inplace=True)

In [27]:
for i, doc in ma.diff_dataset.iterrows():
    prediction = mt.dataset_withoutNAN[(mt.dataset_withoutNAN.home == doc.home) & (mt.dataset_withoutNAN.away == doc.away)].pred
    if len(prediction)>0:
        ma.diff_dataset.at[i, 'prediction'] = prediction.values[0]

In [28]:
for i, doc in ma.diff_dataset.iterrows():
    prediction = mt.dataset_withoutNAN[(mt.dataset_withoutNAN.home == doc.home) & (mt.dataset_withoutNAN.away == doc.away)].pred
    if len(prediction) > 0:
        ma.diff_dataset.at[i, 'prediction'] = prediction.values[0]
    else:
        ma.diff_dataset.at[i, 'prediction'] = -1

In [29]:
ma.diff_dataset.to_csv("dataseta_completo.csv")

In [32]:
c1 = Classification(ma.X_train, ma.X_test, ma.y_train, ma.y_test) 

In [33]:
c1.create_models()
c1.gen_report()
c1.get_higher_accuracies()

ValueError: could not convert string to float: 'P'